# Registers

In memory mapped register ('MMR') decoders there is a need for a certain abstraction in order to maintain adressable registers.

In [11]:
from cyhdl import *
from myirl.library.registers import *

## Bitfield and Register classes

A Register contains a set of bit fields which can have a specific value. For bus decoding, it is often necessary to map a register item to a plain data bus signal.

In [12]:
BF = NamedBitfield
A = BF("im", 3, 1)
B = BF("ex", 7, 6)
C = BF("inv", 4, 4)
D = BF("mode", 14, 10)

r = Register(16, [A, B, C, D])

Dump bit map in MSB->LSB order. Bits listed as `None` are undefined. When reading from a MMR, undefined values may be not asserted to the bus, or assigned to a default value, depeding on the inference rules.

In [13]:
list(reversed(r.layout))

[[None 15:15],
 [mode 14:10],
 [None 9:8],
 [ex 7:6],
 [None 5:5],
 [inv 4:4],
 [im 3:1],
 [None 0:0]]

### Concat composition

To return a concatenation, we use the `.compose` notation below.
We first instance a dummy target module (VHDL by default) to elaborate on the HDL equivalent.

In [14]:
from myirl.targets.dummy import DummyTargetModule

d = DummyTargetModule()

In [15]:
s = Signal(intbv()[3:], name = 's')

c = r.compose(im = s, ex = 3, mode = 5)

q = Signal(intbv()[16:], name = 'q')
a = q.set(c)

a.emit(d)

q <= ('0' & "00101" & "00" & "11" & '0' & '0' & s & '0');


### Partial assignment

The partial assignment is not supported with standard `Signal` types, a `PASignal` must be used instead for the alternative composition, returning generators. Note that a PASignal can **not** be passed through the interface.
Also note that assigning PASignal slices directly (such as `pa[5:3].set(5)` is not fully portable. The proper coding practise is to use PAsignals together in combination with the `register.assign()` method listed below.

In [16]:
p = PASignal(intbv()[16:], name = 'p')

In [17]:
gen = r.assign(p, im = s, ex = 3, mode = 5)

In [18]:
gen.emit(d)

p(3 downto 1) <= s;
p(7 downto 6) <= "11";
p(14 downto 10) <= "00101";


### Issues

Partial assignments cause some issues in internal driver resolving and can under some circumstances create more logic congestion than actually necessary.

The recommended approach is to use the `.compose` method to explicitely assign all bits of a signal. For the register class, the `.undefined` method can be overriden to use other default values:

In [19]:
from myirl.library.dontcare import DontCare

class MyRegister(Register):
    def undefined(self, size):
        return DontCare(size, '?')

In [20]:
r = MyRegister(16, [A, B, C, D])
s = Signal(intbv()[3:], name = 's')

vlog = DummyTargetModule(targets.Verilog)
c = r.compose(im = s, ex = 3, mode = 5)
q.set(c).emit(vlog)

assign  q = {1'b?, 5'b00101, 2'b??, 2'b11, 1'b?, 1'b0, s, 1'b?}; /* fallback */
